In [ ]:
import vtk
from vtk.util.numpy_support import vtk_to_numpy

from scipy import ndimage
from time import clock

import gdal
import meshio
import numpy as np
import pandas as pd
from netCDF4 import Dataset

## Reading eTOPO dataset...

In [ ]:
filename = "data/ETOPO1.tif"
gtiff = gdal.Open(filename)
img = np.fliplr(gtiff.GetRasterBand(1).ReadAsArray().T)

## Reading Scotese dataset...

In [ ]:
data = Dataset("data/scotese/5Ma.nc", "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

## Read mesh from refinement level

In [ ]:
ref_lvl = 10
meshFile = "data/globe_ref"+str(ref_lvl)

Extract mesh information from `vtk` file:

In [ ]:
t0 = clock()
mdata = meshio.read(meshFile+".vtk")

# Information...
cells = np.asarray(mdata.cells['triangle'], dtype=np.int32)
coords = np.asarray(mdata.points, dtype=np.double)
ngbhs = np.asarray(mdata.point_data['ngbhs'], dtype=np.double)
print(clock()-t0)

From `XYZ` coordinates to `lon/lat`:

In [ ]:
radius = 6378137.0
lats = np.arcsin(coords[:,-1]/radius)
lons = np.arctan2(coords[:,1],coords[:,0]) 

Convert spherical mesh longitudes and latitudes to degrees

In [ ]:
dlons = np.mod(np.degrees(lons)+180.0, 360.0)
dlats = np.mod(np.degrees(lats)+90, 180.0)

## Map mesh coordinates on ETOPO

In [ ]:
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

Define by interpolation eTOPO values...

In [ ]:
elev = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

## Store compressed numpy file

Create a numpy compressed `npz` file...

In [ ]:
meshAge = "data/scotese5Ma"+str(ref_lvl)
np.savez_compressed(meshAge, v=coords, n=ngbhs, z=elev)

## Making it as a function....

In [ ]:
def paleoElevation(elevfile,modelfile,ref_lvl,paleofile):
    
    if elevfile.endswith('.tif'):
        gtiff = gdal.Open(elevfile)
        img = np.fliplr(gtiff.GetRasterBand(1).ReadAsArray().T)
    elif elevfile.endswith('.nc'):
        data = Dataset(elevfile, "r", format="NETCDF4")
        img = np.fliplr(data['z'][:,:].T)
    else:
        print(elevfile, "is an unknown file format.")
        print("Accepted formats: geotif (.tif) and netCDF4 (.nc).")
        return
    
    meshFile = modelfile+str(ref_lvl)
    mdata = meshio.read(meshFile+".vtk")

    cells = np.asarray(mdata.cells['triangle'], dtype=np.int32)
    coords = np.asarray(mdata.points, dtype=np.double)
    ngbhs = np.asarray(mdata.point_data['ngbhs'], dtype=np.double)
    
    radius = 6378137.0
    lats = np.arcsin(coords[:,-1]/radius)
    lons = np.arctan2(coords[:,1],coords[:,0]) 
    
    dlons = np.mod(np.degrees(lons)+180.0, 360.0)
    dlats = np.mod(np.degrees(lats)+90, 180.0)
    
    ilons = img.shape[0] * dlons / 360.0
    ilats = img.shape[1] * dlats / 180.0
    icoords = np.stack((ilons, ilats))
    
    elev = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)
    meshAge = paleofile+str(ref_lvl)
    np.savez_compressed(meshAge, v=coords, c=cells, n=ngbhs, z=elev)
    
    print('Your paleo-elevation file has been created: ',meshAge+'.npz')
    
#     mdata = meshio.read(meshFile+".vtk")
#     mesh1 = meshio.Mesh(coords,cells={"triangle": cells},point_data={"z":elev}) 
#     meshio.write("test_elev.vtk", mesh1)
    
    return

In [ ]:
elevfile = "data/scotese/60Ma.nc"
modelfile = "data/globe_ref"
ref_lvl = 10
paleofile = "data/scotese60Ma"

paleoElevation(elevfile,modelfile,ref_lvl,paleofile)

## Velocity maps 

In [ ]:
def paleoVelocity(velfile,modelfile,ref_lvl,paleovelo):
    
    data = pd.read_csv(velfile, sep=r'\s+', engine='c', header=None, skiprows=[0,1,2,3,4,5,65166], 
                       error_bad_lines=True, na_filter=False, dtype=np.float, low_memory=False)

    lon = data.values[:,0]
    lat = data.values[:,1]

    # conversion cm/yr to m/yr
    tmpx = data.values[:,2]/100.
    tmpy = data.values[:,3]/100.
    tmpz = data.values[:,4]/100.

    tmpx = np.fliplr(tmpx.reshape((181,360)).T)
    tmpy = np.fliplr(tmpy.reshape((181,360)).T)
    tmpz = np.fliplr(tmpz.reshape((181,360)).T)

    dispX = np.zeros((361,181))
    dispY = np.zeros((361,181))
    dispZ = np.zeros((361,181))

    dispX[:360,:] = tmpx
    dispX[-1,:] = tmpx[0,:]
    dispY[:360,:] = tmpy
    dispY[-1,:] = tmpy[0,:]
    dispZ[:360,:] = tmpz
    dispZ[-1,:] = tmpz[0,:]

    meshFile = modelfile+str(ref_lvl)
    mdata = meshio.read(meshFile+".vtk")

    cells = np.asarray(mdata.cells['triangle'], dtype=np.int32)
    coords = np.asarray(mdata.points, dtype=np.double)
    ngbhs = np.asarray(mdata.point_data['ngbhs'], dtype=np.double)

    radius = 6378137.0
    lats = np.arcsin(coords[:,-1]/radius)
    lons = np.arctan2(coords[:,1],coords[:,0]) 

    dlons = np.mod(np.degrees(lons)+180.0, 360.0)
    dlats = np.mod(np.degrees(lats)+90, 180.0)

    ilons = dispX.shape[0] * dlons / 360.0
    ilats = dispX.shape[1] * dlats / 180.0
    icoords = np.stack((ilons, ilats))

    dX = ndimage.map_coordinates(dispX , icoords, order=3, mode='nearest').astype(np.float)
    dY = ndimage.map_coordinates(dispY , icoords, order=3, mode='nearest').astype(np.float)
    dZ = ndimage.map_coordinates(dispZ , icoords, order=3, mode='nearest').astype(np.float)

    disps = np.stack((dX, dY, dZ)).T

    velAge = paleovelo+str(ref_lvl)
    np.savez_compressed(velAge, xyz=disps)

    print('Your paleo-velocity file has been created: ',velAge+'.npz')
    
#     mdata = meshio.read(meshFile+".vtk")
#     mesh1 = meshio.Mesh(coords,cells={"triangle": cells},point_data={"xyz":disps}) 
#     meshio.write("test_disp.vtk", mesh1)
    
    return

In [ ]:
for k in range(10):

    velfile = "data/velocityGlobal/velocity_"+str(k)+".00Ma.xy"
    modelfile = "data/globe_ref"
    ref_lvl = 10
    paleovelo = "data/vel"+str(k)+"Ma"

    paleoVelocity(velfile,modelfile,ref_lvl,paleovelo)

## Testing other stuff...

Testing extracting information timing...

In [ ]:
t0 = clock()
loaded = np.load(meshAge+".npz")
coords = loaded['v']
ngbhs = loaded['n']
elev = loaded['z']

print(clock()-t0)
print(elev.shape,ngbhs.shape,coords.shape)